In [3]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()

llm = ChatOpenAI(model='gpt-4o')
small_llm = ChatOpenAI(model='gpt-4o-mini')

In [4]:
from langchain_core.tools import tool

@tool 
def add(a: int, b:int) -> int:
    """숫자 a와 b를 더합니다.""" 
    return a + b

@tool
def multiply(a: int, b:int) -> int:
    """숫자 a와 b를 곱합니다."""
    return a * b

In [5]:
from langchain_community.tools import DuckDuckGoSearchRun

web_search_tool = DuckDuckGoSearchRun()

#web_search_tool.invoke("오바마가 태어난 곳의 화폐 단위는?")

In [6]:
import inspect
from langchain_google_community.gmail.utils import get_gmail_credentials

inspect.signature(get_gmail_credentials)

<Signature (token_file: 'Optional[str]' = None, client_sercret_file: 'Optional[str]' = None, service_account_file: 'Optional[str]' = None, scopes: 'Optional[List[str]]' = None, use_domain_wide: 'bool' = False, delegated_user: 'Optional[str]' = None) -> 'Credentials'>

In [7]:
from langchain_google_community import GmailToolkit
from langchain_google_community.gmail.utils import (
    build_resource_service,
    get_gmail_credentials,
)

# Can review scopes here https://developers.google.com/gmail/api/auth/scopes
# For instance, readonly scope is 'https://www.googleapis.com/auth/gmail.readonly'
credentials = get_gmail_credentials(
    token_file="../google/token.json", #나중에 만들어지는 파일 
    scopes=["https://mail.google.com/"],
    client_sercret_file="../google/credentials.json",
)

api_resource = build_resource_service(credentials=credentials)
gmail_toolkit = GmailToolkit(api_resource=api_resource)
gmail_tool_list = gmail_toolkit.get_tools()

#app publish, gmail API 승인 필요 

/var/folders/h2/84bkz1z16w5c0lv50z7fjm_40000gn/T/ipykernel_72262/1023507223.py:9: DeprecationWarning: get_gmail_credentials is deprecated and will be removed in a future version.Use get_google_credentials instead.
  credentials = get_gmail_credentials(
/var/folders/h2/84bkz1z16w5c0lv50z7fjm_40000gn/T/ipykernel_72262/1023507223.py:15: DeprecationWarning: build_resource_service is deprecated and will be removed in a future version.Use build_gmail_service instead.
  api_resource = build_resource_service(credentials=credentials)


In [8]:
from langchain_community.tools.arxiv.tool import ArxivQueryRun
from langchain_community.utilities.arxiv import ArxivAPIWrapper

arxiv_tool = ArxivQueryRun(api_wrapper=ArxivAPIWrapper())

In [9]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.tools.retriever import create_retriever_tool

embeddings = OpenAIEmbeddings(model='text-embedding-3-large')
vector_store = Chroma(
    embedding_function=embeddings,
    collection_name = 'real_estate_tax',
    persist_directory='../real_estate_tax_collection'
)

retriever = vector_store.as_retriever(search_kwargs={'k': 3})

retriever_tool = create_retriever_tool(
    retriever=retriever,
    name='real_estate_tax_retriever',
    description="2024년 12월까지의 부동산 세금(취득세, 보유세, 양도소득세 등) 정보를 포함하고 있습니다. 부동산 세금 관련 법령이나 계산 방식에 대한 질문이 들어오면 이 도구를 사용하세요.",
)

In [10]:
from langgraph.prebuilt import ToolNode

#함수 객체 리스트 정의 
tool_list = [add, multiply, web_search_tool, arxiv_tool, retriever_tool] + gmail_tool_list

#모델에 도구 설명서 전달 
#llm_with_tools = small_llm.bind_tools(tool_list)
llm_with_tools = llm.bind_tools(tool_list)

#함수 객체 리스트를 전달받아, llm의 도구 호출 지시에 따라 실제로 파이썬 함수를 실행하는 노드를 생성
tool_node = ToolNode(tool_list)

In [11]:
from langgraph.graph import MessagesState, StateGraph

# graph_builder = StateGraph(AgentState)
graph_builder = StateGraph(MessagesState)

In [12]:
def agent(state: MessagesState) -> MessagesState:
    """
    에이전트 함수는 주어진 상태에서 메시지를 가져와
    LLM과 도구를 사용하여 응답 메시지 (AIMessage)를 생성합니다.

    Args:
        state (MessagesState): 메시지 상태를 포함하는 state.

    Returns:
        MessagesState: 응답 메시지를 포함하는 새로운 state.
    """
    # 상태에서 메시지 리스트를 추출합니다.
    messages = state['messages']
    
    # LLM과 도구를 사용하여 메시지를 처리하고 응답을 생성합니다.
    response = llm_with_tools.invoke(messages)
    
    # 응답 메시지를 새로운 상태로 반환합니다.
    return {'messages': [response]}

In [13]:
graph_builder.add_node('agent', agent)
graph_builder.add_node('tools', tool_node)

In [14]:
from langgraph.graph import START, END
from langgraph.prebuilt import tools_condition

graph_builder.add_edge(START, 'agent')

graph_builder.add_conditional_edges(
    'agent',
    tools_condition,
)
graph_builder.add_edge('tools', 'agent')

In [15]:
from langgraph.checkpoint.memory import MemorySaver

# 1. 기억 저장소(창고) 객체를 만듭니다.
checkpointer = MemorySaver()

# 2. 그래프를 컴파일할 때 이 저장소를 연결합니다.
graph = graph_builder.compile(
    checkpointer = checkpointer
)

In [16]:
from langchain.messages import HumanMessage

# 3. 특정 대화방을 구분할 번호표(thread_id)를 정합니다.
config = {
    'configurable': {
        'thread_id': 'paper_summary_01'  # 대화방 별 고유 이름
    }
}

query = "Attention is All You Need라는 논문을 요약해서 leehnjee@gmail.com으로 보내기 위한 초안을 작성해주세요."

# 4. 실행할 때 이 config를 함께 전달합니다.
for chunk in graph.stream({'messages': [HumanMessage(query)]}, config=config, stream_mode='values'):
    chunk['messages'][-1].pretty_print()

================================ Human Message =================================

Attention is All You Need라는 논문을 요약해서 leehnjee@gmail.com으로 보내기 위한 초안을 작성해주세요.
================================== Ai Message ==================================
Tool Calls:
  arxiv (call_fmCrkUiYVBUHUyFQX1vDSYmv)
 Call ID: call_fmCrkUiYVBUHUyFQX1vDSYmv
  Args:
    query: Attention is All You Need
  create_gmail_draft (call_YKNaSMgp5c9NdTfA5MZva2RK)
 Call ID: call_YKNaSMgp5c9NdTfA5MZva2RK
  Args:
    message: 이 이메일은 NLP 분야에서 Transformer 구조의 기초를 설명한 논문 'Attention is All You Need'의 요약을 포함하고 있습니다. Transformer는 주로 셀프 어텐션 메커니즘을 활용하여 번역, 텍스트 생성 등의 작업에서 혁신적인 성과를 보여주었습니다. 이 논문은 구체적으로 어텐션 메커니즘이 어떠한 방식으로 작동하는지, 그리고 기존 구조들 대비 어떠한 이점이 있는지를 설명합니다.
    to: ['leehnjee@gmail.com']
    subject: Attention is All You Need 논문 요약
================================= Tool Message =================================
Name: create_gmail_draft

Draft created. Draft Id: r2712269448353513460
================================== Ai Message ====

In [19]:
updated_query = "초안에 논문의 출처 url도 추가해주세요."

for chunk in graph.stream({'messages': [HumanMessage(updated_query)]}, config=config, stream_mode='values'): #paper_summary_01 대화방에 재질문 
    chunk['messages'][-1].pretty_print()


================================ Human Message =================================

초안에 논문의 출처 url도 추가해주세요.
================================== Ai Message ==================================
Tool Calls:
  arxiv (call_jfo22Ohn8k6ztv5m2o6Vkf0K)
 Call ID: call_jfo22Ohn8k6ztv5m2o6Vkf0K
  Args:
    query: Attention is All You Need site:arxiv.org
================================= Tool Message =================================
Name: arxiv

Published: 2021-05-06
Title: Do You Even Need Attention? A Stack of Feed-Forward Layers Does Surprisingly Well on ImageNet
Authors: Luke Melas-Kyriazi
Summary: The strong performance of vision transformers on image classification and other vision tasks is often attributed to the design of their multi-head attention layers. However, the extent to which attention is responsible for this strong performance remains unclear. In this short report, we ask: is the attention layer even necessary? Specifically, we replace the attention layer in a vision transformer with 

Impersonate 'chrome_124' does not exist, using 'random'


================================== Ai Message ==================================
Tool Calls:
  duckduckgo_search (call_2upxMnem4h7xTziR86GvowuB)
 Call ID: call_2upxMnem4h7xTziR86GvowuB
  Args:
    query: Attention is All You Need arxiv link
================================= Tool Message =================================
Name: duckduckgo_search

2025. 10. 21. · Abstract:We demonstrate complete functional segregation in hybrid SSM-Transformer architectures: retrieval depends exclusively on self-attention layers. 2025. 5. 29. · tuning을 하지않았음에도 좋은 성능을 확인. 참고 링크. Vaswani, A., et al. (2017). Attention Is All You Need. arXiv:1706.03762. https://arxiv.org/abs ... 2025. 4. 5. · "Attention Is All You Need"는 Transformer 모델을 제안한 기념비적인 논문으로, 자연어 처리 분야에 혁신을 일으킨 연구이다. 기존의 순환 신경망(RNN) 기반 모델들과 ... 2025. 8. 3. · ATTENTION IS ALL YOU NEED Original Paper by Ashish Vaswani, Noam Shazeer ... Original Paper link- https://arxiv.org/pdf/1706.03762. Table of Contents. 1 ... 2025. 12. 12. · Abstract page for arX

In [20]:
updated_query = "좋습니다. 이 초안을 전송해주세요."

for chunk in graph.stream({'messages': [HumanMessage(updated_query)]}, config=config, stream_mode='values'): #paper_summary_01 대화방에 재질문 
    chunk['messages'][-1].pretty_print()

================================ Human Message =================================

좋습니다. 이 초안을 전송해주세요.
================================== Ai Message ==================================
Tool Calls:
  send_gmail_message (call_tshfQ7XEuwQMxSCO2CfhKXqF)
 Call ID: call_tshfQ7XEuwQMxSCO2CfhKXqF
  Args:
    message: 이 이메일은 NLP 분야에서 Transformer 구조의 기초를 설명한 논문 'Attention is All You Need'의 요약을 포함하고 있습니다. Transformer는 주로 셀프 어텐션 메커니즘을 활용하여 번역, 텍스트 생성 등의 작업에서 혁신적인 성과를 보여주었습니다. 이 논문은 구체적으로 어텐션 메커니즘이 어떠한 방식으로 작동하는지, 그리고 기존 구조들 대비 어떠한 이점이 있는지를 설명합니다.

논문의 출처: [Attention Is All You Need](https://arxiv.org/abs/1706.03762)
    to: ['leehnjee@gmail.com']
    subject: Attention is All You Need 논문 요약
================================= Tool Message =================================
Name: send_gmail_message

Message sent. Message Id: 19c8f923cf2cf2b4
================================== Ai Message ==================================

The email with the summary of "Attention is All You Need" has been successfully se

In [24]:
message_list = graph.get_state(config).values['messages']
message_list

[HumanMessage(content='Attention is All You Need라는 논문을 요약해서 leehnjee@gmail.com으로 보내기 위한 초안을 작성해주세요.', additional_kwargs={}, response_metadata={}, id='bfb5265a-8cdb-4d72-b04a-b31b5f4576ff'),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 180, 'prompt_tokens': 818, 'total_tokens': 998, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_64dfa806c7', 'id': 'chatcmpl-DClZgfZ27yevCIsm2s6IIGPzxTQC9', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c8f91-32e9-7721-9d00-cc962cc7ae8d-0', tool_calls=[{'name': 'arxiv', 'args': {'query': 'Attention is All You Need'}, 'id': 'call_fmCrkUiYVBUHUyFQX1vDSYmv', 'type': 'tool_call'}, {'name': 'create_gmai

### 첫 질문에 대한 그래프 흐름 정리해보기 
1. START -> agent 노드
    - MessageState: [HumanMessage]
    - 행동: LLM이 "도구가 필요해!"라고 판단하여 tool_calls가 담긴 AIMessage 생성.

2. agent -> tools 노드 
    - **AIMessage에 tool_calls가 있으므로 tools_condition에 의해 tools 노드로 이동**
    - MessageState: [HumanMessage, AIMessage(tool_calls)]
    - 행동: arxiv, search_gmail 실행 후 결과값을 ToolMessage로 반환.

3. tools -> agent 노드 (무조건 연결된 edge)
    - MessageState: [HumanMessage, AIMessage, ToolMessage, ToolMessage]
    - 행동: 도구 결과를 본 LLM이 "오케이, 이제 초안을 만들자" 하고 create_gmail_draft를 위한 두 번째 AIMessage 생성.

4. agent -> tools -> agent (한 번 더 반복)
    - 이번 AIMessage에도 tool_calls가 있으므로 tools_condition에 의해 tools 노드로 이동, tools 노드에서 ToolMessage 리턴
    - 상태: 메시지 리스트가 점점 길어짐.
    - 행동: 초안 생성 결과(ToolMessage)를 확인한 LLM이 최종 답변을 준비.

5. agent -> END (tools_condition에 의해)
    - 상태: 모든 메시지가 쌓인 상태.
    - 행동: 모든 메시지 리스트를 검토한 LLM이 더 이상 tool_calls 없이 content만 채운 마지막 AIMessage 반환. 
        - **tools_condition은 "더 실행할 도구 없네?" 하고 END로 보냄.**